### Dataframe style looks better

In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import os
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/speech_commands' # to concatenate audios
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Command_Dataframe.pkl')

In [3]:
all_spks = list(data_df.spk.unique())
all_words = list(data_df.sent.unique())

## Manifest For Trial

### SV DataFrame

In [42]:
spk_counts = data_df.spk.value_counts()

audio를 30개 이상 가진 스피커를 enroll speaker로 둔다.

In [53]:
sv_candidate = random.choice(spk_counts[spk_counts == 30].index, size=500, replace=False)

In [56]:
sv_val_spks = sv_candidate[:250]
sv_test_spks = sv_candidate[250:]

In [57]:
sv_val_df = data_df[data_df.spk.isin(sv_val_spks)]
sv_test_df = data_df[data_df.spk.isin(sv_test_spks)]

In [58]:
print("number of sv_val spks: {}".format(len(sv_val_df)))
print("number of sv_test spks: {}".format(len(sv_test_df)))

number of sv_val spks: 7500
number of sv_test spks: 7500


trial이 저장될 폴더

In [59]:
dir_name = "trials/commands/validation/"
if not os.path.isdir(dir_name):
    os.makedirs(dir_name)

In [60]:
sv_val_df.to_pickle("{}/sv_val.pkl".format(dir_name))
sv_test_df.to_pickle("{}/sv_test.pkl".format(dir_name))

### SI train Dataframe 

In [61]:
si_df = data_df[~data_df.spk.isin(sv_candidate)]

In [63]:
si_df.to_pickle("{}/si_train.pkl".format(dir_name))

In [64]:
print("number of si spks: {}".format(len(si_df)))

number of si spks: 49721


## For Human Performance

enrollment speakers: 3  
enrollment audios: 3  
pos/neg test samples: 5, 5

In [249]:
len(sv_test_df)

7500

In [278]:
n_enroll_spks = 3
n_enroll_uttrs = 2
n_pos_test_samples = 20
n_neg_test_samples = 300

In [279]:
h_enroll_spks = np.random.choice(sv_test_spks, size=n_enroll_spks, replace=False)

In [280]:
samples_dict = dict()
imposter_uttrs = sv_test_df[~(sv_test_df.spk.isin(h_enroll_spks))].sample(n=n_neg_test_samples)
imposter_uttrs['set'] = 'neg_test'
imposter_uttrs['spk'] = 'imposter'
samples_dict['imposter'] = imposter_uttrs
for spk in h_enroll_spks:
    target_uttrs = sv_test_df[sv_test_df.spk == spk].sample(n=n_enroll_spks+n_pos_test_samples)
    target_uttrs.iloc[:n_enroll_uttrs]['set'] = 'enroll'
    target_uttrs.iloc[n_enroll_uttrs:]['set'] = 'pos_test'
    samples_dict[spk] = target_uttrs

/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [281]:
total_samples = pd.concat([df for df in samples_dict.values()])

In [282]:
total_samples.to_pickle("trials/human_perf/samples.pkl")

In [283]:
from pydub import AudioSegment
data_folder = "/home/muncok/DL/dataset/SV_sets/speech_commands/"
result_folder = "trials/human_perf/"

for spk in list(h_enroll_spks) + ['imposter']:
    i = 0
    r_folder = os.path.join(result_folder, spk)
    if not os.path.isdir(r_folder):
        os.makedirs(r_folder)
    for idx, row in samples_dict[spk].iterrows():
        s = AudioSegment.from_wav(os.path.join(data_folder, row.sent, row.file))
        s.export(os.path.join(r_folder, "{}_{}.wav".format(row.set, i)), format='wav')
        i+=1

In [266]:
pd.read_pickle("./trials")

,spk,sent,file,set,label
dog839572010,imposter,dog,83957201_nohash_0.wav,neg_test,435
righte3b642170,imposter,right,e3b64217_nohash_0.wav,neg_test,1262
wowdabf67d90,imposter,wow,dabf67d9_nohash_0.wav,neg_test,83
yes8d9cc2600,imposter,yes,8d9cc260_nohash_0.wav,neg_test,1556
stopd1a4fb3f0,imposter,stop,d1a4fb3f_nohash_0.wav,neg_test,1593
yesc25a032b0,imposter,yes,c25a032b_nohash_0.wav,neg_test,530
birde65154150,imposter,bird,e6515415_nohash_0.wav,neg_test,453
onf9273a211,imposter,on,f9273a21_nohash_1.wav,neg_test,388
treebbb2eb5b0,imposter,tree,bbb2eb5b_nohash_0.wav,neg_test,950
wowe3e499310,imposter,wow,e3e49931_nohash_0.wav,neg_test,967
